In [2]:
import numpy as np
import pandas as pd
import zipfile

import matplotlib.pyplot as plt
import seaborn as sns
import math 

/Applications/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


# 1. Import Data

In [3]:
ztrain = zipfile.ZipFile('train.csv.zip')
ztest = zipfile.ZipFile('test.csv.zip')

train = pd.read_csv(ztrain.open('train.csv'))
test = pd.read_csv(ztest.open('test.csv'))


In [4]:
train.info()
train.tail(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29118021 entries, 0 to 29118020
Data columns (total 6 columns):
row_id      int64
x           float64
y           float64
accuracy    int64
time        int64
place_id    int64
dtypes: float64(2), int64(4)
memory usage: 1.3 GB


,row_id,x,y,accuracy,time,place_id
29118016,29118016,6.5133,1.1435,67,399740,8671361106
29118017,29118017,5.9186,4.4134,67,125480,9077887898
29118018,29118018,2.9993,6.3680,67,737758,2838334300
29118019,29118019,4.0637,8.0061,70,764975,1007355847
29118020,29118020,7.4523,2.0871,17,102842,7028698129


In [5]:
test.info()
test.tail(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8607230 entries, 0 to 8607229
Data columns (total 5 columns):
row_id      int64
x           float64
y           float64
accuracy    int64
time        int64
dtypes: float64(2), int64(3)
memory usage: 328.3 MB


,row_id,x,y,accuracy,time
8607225,8607225,4.1206,5.2443,58,882527
8607226,8607226,5.1170,5.7695,60,984517
8607227,8607227,6.6409,8.3626,19,814024
8607228,8607228,6.4190,2.9985,68,862916
8607229,8607229,8.1017,7.8736,1,845096


In [6]:
train.loc[:,'row_id'] = train.row_id + 8607230

In [9]:
train = train.set_index('row_id',drop=True)
test = test.set_index('row_id',drop=True)

train_index = train.index.values
test_index = test.index.values

concat_all = pd.concat([train.drop('place_id',axis=1),test],ignore_index=0)
concat_all.info()
concat_all.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37725251 entries, 8607230 to 8607229
Data columns (total 4 columns):
x           float64
y           float64
accuracy    int64
time        int64
dtypes: float64(2), int64(2)
memory usage: 1.4 GB


,x,y,accuracy,time
row_id,,,,
8607230,0.7941,9.0809,54,470702
8607231,5.9567,4.7968,13,186555
8607232,8.3078,7.0407,74,322648
8607233,7.3665,2.5165,65,704587
8607234,4.0961,1.1307,31,472130


# 2. Data Preparation

### Check for incorrect values

# 3.  Model Building

In [ ]:
# create MAP@3 function for grid search
# https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/average_precision.py

def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=3):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])